In [1]:
from os.path import getsize
from xdr_tools import *
from _vacuumtoair import *
import h5py

In [2]:
__XDR_Specs__ = dict()
__XDR_Specs__['atmos_3d_input'] = XDR_Specs(
    [
        ['Nx', int, None],
        ['Ny', int, None],
        ['Nz', int, None],
        ['NHydr', int, None],
        ['Top', int, None],
        ['Bot', int, None],
        ['dx', float, None],
        ['dy', float, None],
        ['z', float, 'Nz'],
        ['T', float, '(Nx,Ny,Nz)'],
        ['ne', float, '(Nx,Ny,Nz)'],
        ['vturb', float, '(Nx,Ny,Nz)'],
        ['vx', float, '(Nx,Ny,Nz)'],
        ['vy', float, '(Nx,Ny,Nz)'],
        ['vz', float, '(Nx,Ny,Nz)'],
        ['nH', float, '(Nx,Ny,Nz,NHydr)']
    ]
)

In [3]:
specs = XDR_Specs(__XDR_Specs__['atmos_3d_input'])

In [4]:
reader = XDR_Reader(specs)
wt = XDR_Struct(specs)

In [5]:
f = h5py.File('/home/harsh/BifrostRun/bifrost_en024048_hion_0_504_0_504_-500000.0_3000000.0.nc', 'r')

In [6]:
f.keys()

<KeysViewHDF5 ['B_x', 'B_y', 'B_z', 'depth', 'electron_density', 'hydrogen_populations', 'nhydr', 'snapshot_number', 'temperature', 'velocity_x', 'velocity_y', 'velocity_z', 'x', 'y', 'z']>

In [7]:
wt['Nx'] = f['x'][()].size

In [8]:
wt['Ny'] = f['y'][()].size

In [9]:
wt['Nz'] = f['z'].shape[-1]

In [10]:
wt['NHydr'] = f['nhydr'][()].size

In [11]:
f.attrs.keys()

<KeysViewHDF5 ['comment', 'boundary_top', 'boundary_bottom', 'has_B', 'description', 'nx', 'ny', 'nz', 'nt', '_NCProperties']>

In [12]:
f.attrs['description']

b'Bifrost Simulation (simulation_name, start_x, end_x, start_y, end_y, height_min_in_m, height_max_in_m) - en024048_hion - 0 - 504 - 0 - 504 - -500000.0 - 3000000.0'

In [13]:
f.attrs['boundary_top'][()][0], f.attrs['boundary_bottom'][()][0]

(0, 1)

In [14]:
wt['Top'] = 1

In [15]:
wt['Bot'] = 2

In [16]:
wt['dx'] = 48.0

In [17]:
wt['dy'] = 48.0

In [18]:
wt['z'] = f['z'][0, 0, 1]/1e3

In [19]:
f['temperature'].shape

(1, 504, 504, 177)

In [20]:
wt['T'] = f['temperature'][0]

In [21]:
wt['ne'] = f['electron_density'][0]

In [22]:
wt['vturb'] = np.zeros((f['x'][()].size, f['y'][()].size, f['z'].shape[-1]))

In [23]:
wt['vx'] = f['velocity_x'][0]/1e3

In [24]:
wt['vy'] = f['velocity_y'][0]/1e3

In [25]:
wt['vz'] = f['velocity_z'][0] / 1e3

In [26]:
wt['nH'] = np.transpose(f['hydrogen_populations'][0], axes=(1, 2, 3, 0))

In [27]:
wt

Nx           <class 'int'>
Ny           <class 'int'>
Nz           <class 'int'>
NHydr        <class 'int'>
Top          <class 'int'>
Bot          <class 'int'>
dx           <class 'float'>
dy           <class 'float'>
z            <class 'numpy.ndarray'>         (177,)
T            <class 'numpy.ndarray'>         (504, 504, 177)
ne           <class 'numpy.ndarray'>         (504, 504, 177)
vturb        <class 'numpy.ndarray'>         (504, 504, 177)
vx           <class 'numpy.ndarray'>         (504, 504, 177)
vy           <class 'numpy.ndarray'>         (504, 504, 177)
vz           <class 'numpy.ndarray'>         (504, 504, 177)
nH           <class 'numpy.ndarray'>         (504, 504, 177, 6)

In [28]:
reader.write('/home/harsh/BifrostRun/bifrost_en024048_hion_0_504_0_504_-500000.0_3000000.0.dat', wt)

In [29]:
atmos_3d = reader.read('/home/harsh/BifrostRun/bifrost_en024048_hion_0_504_0_504_-500000.0_3000000.0.dat')